<a href="https://colab.research.google.com/github/dimitarpg13/langchain_tutorial/blob/main/langchain_tutorial/notebooks/runtime_config/ConfigurableLLMandSystemMessage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Working with Runtime Configurations

We want to configure our graph when calling it. For example, we want to specify LLM or system prompt to use at runtime without polluting the graph state with these parameters.

Adding runtime configuration, invloves a) specifying a schema for the configuration, b) adding the configuration to the nodes or conditional edges and c) passing the configuration to the graph

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph==0.6.0a2 langchain langchain_core  langchain_openai langchain-anthropic dotenv

Specify the necessary keys

In [2]:
import os, getpass
from dotenv import load_dotenv

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

load_dotenv()

_set_env("OPENAI_API_KEY")

_set_env("ANTHROPIC_API_KEY")

## Speciyfing LLM at runtime

Below is a simple example in which we configure two parameters at runtime using the Runtime Configuration functionality: the LLM and system message

In [7]:
from dataclasses import dataclass

from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, StateGraph, START, MessagesState
from typing_extensions import TypedDict
from langchain_core.messages import SystemMessage

from langchain.chat_models import init_chat_model
from langgraph.runtime import Runtime

Specify the context schema

In [8]:
@dataclass
class ContextSchema:
    model_provider: str = "anthropic"
    system_message: str | None = None

MODELS = {
    "anthropic": init_chat_model("anthropic:claude-3-5-haiku-latest"),
    "openai": init_chat_model("openai:gpt-4.1-mini")
}

define call_model and construct the graph

In [14]:
def call_model(state: MessagesState, runtime: Runtime[ContextSchema]):
    model = MODELS[runtime.context['model_provider']]
    messages = state["messages"]
    if (system_message := runtime.context['system_message']):
        messages = [SystemMessage(system_message)] + messages
    response = model.invoke(messages)
    return {"messages": [response]}

builder = StateGraph(MessagesState, context_schema=ContextSchema)
builder.add_node("model", call_model)
builder.add_edge(START, "model")
builder.add_edge("model", END)

graph = builder.compile()

User the graph with test input message

In [15]:
input_message = {"role": "user", "content": "hi"}
response = graph.invoke({"messages": [input_message]}, context={"model_provider": "openai", "system_message": "Respond in Italian."})
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

hi
================================== Ai Message ==================================

Ciao! Come posso aiutarti oggi?
